In [1]:
from dataloader import *

In [2]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

C:\Users\pkgau\anaconda3\Lib\site-packages


In [3]:
import json
config_data = json.load(open('default.json', 'r'))

In [4]:
vocab, train_data_loader, val_data_loader, test_data_loader = get_datasets(config_data)

Using the saved vocab.


C:\Users\pkgau\anaconda3\lib\site-packages\torchvision\transforms\transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [5]:
train_data_loader

In [6]:
for i, (images, ingt, title, ing, ins, img_id) in enumerate(train_data_loader):
    break

In [22]:
images.shape

torch.Size([64, 3, 256, 256])

In [8]:
title.shape

torch.Size([64, 13])

In [9]:
ing.shape

torch.Size([64, 127])

In [10]:
ins.shape

torch.Size([64, 539])

In [11]:
img_id

('17eaddfbf3.jpg',
 '8c9b0fb748.jpg',
 'd3c3a8c33b.jpg',
 '39397c56db.jpg',
 'acaa9bb2de.jpg',
 'c9bd3de7bb.jpg',
 '11879b5362.jpg',
 '4475da7c07.jpg',
 '0ee70edb8e.jpg',
 '9d65cefbf7.jpg',
 '7e6cafe6e1.jpg',
 '324f10bd2a.jpg',
 '9f272b3cc2.jpg',
 'e123f2702a.jpg',
 'fd69b3fc8c.jpg',
 '680d93fbfb.jpg',
 'c8f885303f.jpg',
 'a87e196a61.jpg',
 '7e5b33c749.jpg',
 '6b0d011184.jpg',
 '03d3295718.jpg',
 '0646b12f6e.jpg',
 'ad5cad0597.jpg',
 '7207d4ad29.jpg',
 '23465ada0d.jpg',
 '454ae660a3.jpg',
 '2cde88454b.jpg',
 'ae3d29a7fa.jpg',
 '40bf460fdd.jpg',
 '4f5468d7ba.jpg',
 '98b04298e2.jpg',
 '1786b6aab2.jpg',
 'd6481a3c14.jpg',
 '89fcacc6d0.jpg',
 '8e027d720d.jpg',
 '0a533953b7.jpg',
 '8ffbcda29d.jpg',
 '57f457efe2.jpg',
 'ce86bc003b.jpg',
 '0e7adef3e9.jpg',
 '3520376567.jpg',
 'fc602a64bf.jpg',
 'f89a238420.jpg',
 '95b1741514.jpg',
 'ef28768663.jpg',
 '0fb3e2b4b6.jpg',
 'd00036ecd7.jpg',
 '2411014fed.jpg',
 'a520954a67.jpg',
 'f378ccb7c3.jpg',
 '68512f4242.jpg',
 '0601f95d53.jpg',
 '3c77b7ea31

In [16]:
find_img_path(img_id[0])

'~/val\\1\\7\\e\\a\\17eaddfbf3.jpg'

In [127]:
channel_mean = torch.zeros(1,3,256,256)
channel_std = torch.zeros(1,3,256,256)
for i in range(3):
    channel_mean[0,i,:,:] = torch.mean(images[:,i,:,:], dim = 0, keepdim=True) 
    channel_std[0,i,:,:] = torch.std(images[:,i,:,:], dim = 0, keepdim=True) 
        

In [133]:
print(channel_mean.shape)
torch.mean(channel_mean, dim = [2, 3], keepdim=False)

torch.Size([1, 3, 256, 256])


tensor([[ 0.3774,  0.1051, -0.1764]])

In [134]:
print(channel_std.shape)
torch.mean(channel_std, dim = [2, 3], keepdim=False)

torch.Size([1, 3, 256, 256])


tensor([[1.1593, 1.1756, 1.1958]])